<a href="https://colab.research.google.com/github/djliden/numerai/blob/main/notebooks/regressions_new_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introduction
This notebook will walk you through the entire process of making a [numerai](numer.ai) submission, from downloading the data to submitting final predictions, all in a google colab notebook. In particular, it will address two challenges:
- handling API keys in a remote environment (colab)
- parsing the large CSV files which, if read all at once, will exceed colab's memory and cause the notebook to crash.

This notebook will implement two models: a basic tabular neural network using `fastai` and a linear regression model using `scikit-learn`.

## 1.1 Installing and Importing Dependencies
First, we install and import the necessary packages. This cell is currently set *not* to print any output; if you run into any issues and need to check for error messages, comment out the `%%capture` line

In [1]:
%%capture
# install
!pip install --upgrade python-dotenv numerapi
!pip install --upgrade "dask[complete]"

# import dependencies
import dask.dataframe as dd
import gc
import os
from dotenv import load_dotenv, find_dotenv
from getpass import getpass
import pandas as pd
import numpy as np
import numerapi
from pathlib import Path
from scipy.stats import spearmanr
import sklearn.linear_model
from tqdm import tqdm

## 1.2 Setting Up numerapi
We will use the [numerapi](https://github.com/uuazed/numerapi) package to access the data and make submissions. For this to work, numerapi needs to use your API keys (which can be obtained [here](https://numer.ai/submit)). We will set up two main ways of passing these API keys to a numerapi instance:
1. Read a `.env` file using the `python-dotenv` package. This will require you to upload a `.env` file (which contains your secret key and should *not* be kept under version control). Using this method means you will not have to directly enter your keys each time you use this notebook, though you will need to re-upload the `.env` file.
2. Manually entering the API keys -- if you don't have access to, or don't want to mess with, your `.env` file.

If you have a `.env` file, upload it to the default working directory, `content`, now. In either case, run the cell below to set up the numerapi instance. See [Appendix A](#app_a) for instructions on generating and downloading a .env file.

In [2]:
# Load the numerapi credentials from .env or prompt for them if not available
def credential():
    dotenv_path = find_dotenv()
    load_dotenv(dotenv_path)

    if os.getenv("NUMERAI_PUBLIC_KEY"):
        print("Loaded Numerai Public Key into Global Environment!")
    else:
        os.environ["NUMERAI_PUBLIC_KEY"] = getpass("Please enter your Numerai Public Key. You can find your key here: https://numer.ai/submit -> ")
    
    if os.getenv("NUMERAI_SECRET_KEY"):
        print("Loaded Numerai Secret Key into Global Environment!")
    else:
        os.environ["NUMERAI_SECRET_KEY"] = getpass("Please enter your Numerai Secret Key. You can find your key here: https://numer.ai/submit -> ")
    
    if os.getenv("NUMERAI_MODEL_ID_REGRESSIONS"):
        print("Loaded Numerai Model ID into Global Environment!")
    else:
        os.environ["NUMERAI_MODEL_ID_REGRESSIONS"] = getpass("Please enter your Numerai Model ID. You can find your key here: https://numer.ai/submit -> ")

credential()
public_key = os.environ.get("NUMERAI_PUBLIC_KEY")
secret_key = os.environ.get("NUMERAI_SECRET_KEY")
model_id = os.environ.get("NUMERAI_MODEL_ID_REGRESSIONS")
napi = numerapi.NumerAPI(verbosity="info", public_id=public_key, secret_key=secret_key)

Loaded Numerai Public Key into Global Environment!
Loaded Numerai Secret Key into Global Environment!
Loaded Numerai Model ID into Global Environment!


You can read up on the functionality of numerapi [here](https://github.com/uuazed/numerapi). You can use it to download the competition data, view other numerai users' public profiles, check submission status, manage your stake, and much more. In this case, we'll only be using it to download competition data and submit predictions.

## 1.3 Downloading Competition Data
In a more structured project, you'll probably want to keep the data in a seprate directory from your scripts etc. You could also link google colab to your google drive and store the data there in order to avoid needing to download and process the data every time. In this case, however, we'll keep everything in `./content`, and download the data fresh each time.

In [3]:
napi.download_current_dataset()

2021-04-10 19:45:31,283 INFO numerapi.base_api: target file already exists


'./numerai_dataset_259.zip'

## 1.4 Generating the Training Sample

If you look at the files we downloaded above, you'll see a `numerai_tournament_data.csv` file and a `numerai_training_data.csv` file. The "tournament" file contains many rows with targets which we can use for validation, so let's extract those and combine them with our training set. Note that this cell saves a new `csv` after combining the training and validation data, so we can avoid the time-consuming parsing process if we run this cell again in the same session.

In [4]:
tourn_file = Path(f'./numerai_dataset_{napi.get_current_round()}/numerai_tournament_data.csv')
train_file = Path(f'./numerai_dataset_{napi.get_current_round()}/numerai_training_data.csv')
processed_pkl = Path('./training_processed.pkl')

def process_current(processed_train_pickle,
                    train_file, tourn_file,
                    pickle_out=True):
    
    if processed_train_pickle.exists():
        print("Loading the pickled training data from file\n")
        training_data = pd.read_pickle(processed_train_pickle)
    else:
        print("Processing training data...\n")
        tourn_iter_csv = dd.read_csv(tourn_file)
        tmp_df = tourn_iter_csv[tourn_iter_csv['data_type'] == 'validation']
        training_data = dd.read_csv(train_file)
        training_data = dd.concat([training_data, tmp_df])
        training_data = training_data.compute()
        training_data.reset_index(drop=True, inplace=True)
        #tourn_iter_csv.close()
        print("Training Dataset Generated! Saving to file ...")
        
        if pickle_out:
            training_data.to_pickle(processed_train_pickle)
        
    feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]
    target_cols = ['target']
    return training_data, feature_cols, target_cols

training_data, feature_cols, target_cols = process_current(processed_pkl,
                                                           train_file,
                                                           tourn_file)

train_idx = training_data.index[training_data.data_type=='train'].tolist()
val_idx = training_data.index[training_data.data_type=='validation'].tolist()

Processing training data...

Training Dataset Generated! Saving to file ...


# 2 Modeling the Data

In this section, we will define our evaluation metrics; run two different models (a linear regression model from `scikit-learn` and a neural network from `fastai`); and generate submission dataframes from those files.

## 2.1 Evaluation Metrics

In this section, we will define two key evaluation metrics used to assess the performance of models before submitting to the tournament. These metrics are:
- Average Spearman Correlation per era: The sum of each era's Spearman correlation divided by the number of eras.
- Sharpe Ratio: The average correlation per era divided by the standard deviation of the correlations per era.

Both are defined in reasonable detail [here](https://wandb.ai/carlolepelaars/numerai_tutorial/reports/How-to-get-Started-With-Numerai--VmlldzoxODU0NTQ). The methods defined below are modified versions of the methods described in that post.

In [5]:
def corr(df: pd.DataFrame) -> np.float32:
    """
    Calculate the correlation by using grouped per-era data
    :param df: A Pandas DataFrame containing the columns "era", "target" and "prediction"
    :return: The average per-era correlations.
    """
    def _score(sub_df: pd.DataFrame) -> np.float32:
        """ Calculate Spearman correlation for Pandas' apply method """
        return spearmanr(sub_df["target"],  sub_df["prediction"])[0]
    corrs = df.groupby("era").apply(_score)
    return corrs.mean() 

def sharpe(df: pd.DataFrame) -> np.float32:
    """
    Calculate the Sharpe ratio by using grouped per-era data
    :param df: A Pandas DataFrame containing the columns "era", "target" and "prediction"
    :return: The Sharpe ratio for your predictions.
    """
    def _score(sub_df: pd.DataFrame) -> np.float32:
        """ Calculate Spearman correlation for Pandas' apply method """
        return spearmanr(sub_df["target"],  sub_df["prediction"])[0]
    corrs = df.groupby("era").apply(_score)
    return corrs.mean() / corrs.std()

## 2.2 Cross Validation Setup

### 2.2.1 Custom Cross Validation Setup

The goal of this section is to set up a "group time series" approach where we specify a certain set of "eras" for training with the last era for validation. We will be training on segments of the validation set.

There are a few ways to do this; I want to write a class that can take the "eras to test on" as input and return CV folds as outout. Perhaps a future refinement would include an argument for number of eras validate on. Perhaps unnecessary given that the "real" task is testing on a single era. Or four eras?

#### Usage

1. Initialize the class with the eras column: `cv = EraCV(training_data.era)`
2. Get splits: `X, y = test.get_splits(valid_start = 80, valid_n_eras = 4, train_n_eras = None)`

The `valid_start` argument identifies the first training era; it takes an integer value. `valid_n_eras` is the number of eras to include in the validation set. `train_n_eras` is the number of eras to include in the training set. `train_n_eras` before `valid_start` are included in the training set. If no argument is passed to `train_n_eras`, all eras from 0 to `valid_start` are included.

A single instance of this class can be used in a loop to generate multiple train/test splits. Assuming you want to keep the number of train and test eras constant, you can just iterate over a list of validation starting eras.

Features such as checking if a given validation era actually exists have not yet been implemented.

In [6]:
class EraCV:
    """Select validation eras and train on previous eras
    provides train/test indices to split data in train/test splits. In
    each split, one or more eras are used as a validation set while the
    specified number of immediately preceding eras are used as a
    training set.
    """

    def __init__(self, eras):
        self.eras = eras
        self.unique_eras = self._era_to_int(self.eras.unique())
        self.eras_int = self._era_to_int(self.eras)
    
    def _era_to_int(self, eras):
        return [int(era[3:]) for era in eras]

    def get_valid_indices(self, valid_start, valid_n_eras):
        self.valid_eras = self.unique_eras[self.unique_eras.index(valid_start):\
                                      self.unique_eras.index(valid_start)+\
                                      valid_n_eras]
        valid_bool = [era in self.valid_eras for era in self.eras_int] 
        self.valid_indices = np.where(valid_bool)

    def get_train_indices(self, valid_start:int, train_start:int = 0,
                          train_stop:int = None):
        train_stop = valid_start if (train_stop is None) else train_stop
        self.train_eras = [era for era in self.unique_eras if era <\
                           valid_start][train_start:train_stop]
        train_bool = [era in self.train_eras for era in self.eras_int]
        self.train_indices = np.where(train_bool)

    def get_splits(self, valid_start:int, valid_n_eras:int, train_start:int,
                   train_stop:int=None):
        self.get_valid_indices(valid_start, valid_n_eras)
        self.get_train_indices(valid_start, train_start, train_stop)
        return self.train_indices[0], self.valid_indices[0]

    def __repr__(self):
       return (f'{self.__class__.__name__}('
               f'last era:{max(self.eras_int)})')

In [7]:
training_data.era.unique()

array(['era1', 'era2', 'era3', 'era4', 'era5', 'era6', 'era7', 'era8',
       'era9', 'era10', 'era11', 'era12', 'era13', 'era14', 'era15',
       'era16', 'era17', 'era18', 'era19', 'era20', 'era21', 'era22',
       'era23', 'era24', 'era25', 'era26', 'era27', 'era28', 'era29',
       'era30', 'era31', 'era32', 'era33', 'era34', 'era35', 'era36',
       'era37', 'era38', 'era39', 'era40', 'era41', 'era42', 'era43',
       'era44', 'era45', 'era46', 'era47', 'era48', 'era49', 'era50',
       'era51', 'era52', 'era53', 'era54', 'era55', 'era56', 'era57',
       'era58', 'era59', 'era60', 'era61', 'era62', 'era63', 'era64',
       'era65', 'era66', 'era67', 'era68', 'era69', 'era70', 'era71',
       'era72', 'era73', 'era74', 'era75', 'era76', 'era77', 'era78',
       'era79', 'era80', 'era81', 'era82', 'era83', 'era84', 'era85',
       'era86', 'era87', 'era88', 'era89', 'era90', 'era91', 'era92',
       'era93', 'era94', 'era95', 'era96', 'era97', 'era98', 'era99',
       'era100', 'er

## 2.3 Linear Regression Model
This model closely follows the tutorial example [here](https://colab.research.google.com/github/numerai/example-scripts/blob/master/making-your-first-submission-on-numerai.ipynb). We will use the `scikit-learn` package, with which we can implement and fit our regression model in just a couple of lines of code.

#### Fitting the Linear Regression Model

In [ ]:
%%time
corrs = []
sharpes = []
era_split = EraCV(eras = training_data.era)
X, y, era = training_data[feature_cols], training_data.target, training_data.era
for valid_era in tqdm(range(200,209)):
    train, test = era_split.get_splits(valid_start = valid_era,
                           valid_n_eras = 4,
                           train_n_eras = 50
                           )
    model = sklearn.linear_model.LinearRegression(n_jobs = -1)
    model.fit(X.iloc[train], y.iloc[train])
    val_preds = model.predict(X.iloc[test])
    eval_df = pd.DataFrame({'prediction':val_preds,
                        'target':y.iloc[test],
                        'era':era.iloc[test]}).reset_index()
    corrs.append(corr(eval_df))
    sharpes.append(sharpe(eval_df))

print(corrs)
print(sharpes)

In [ ]:
models = [
          # sklearn.linear_model.LinearRegression(n_jobs = -1),
          # sklearn.linear_model.Lasso(alpha=0.00006), # good! Takes pretty long time.
          # sklearn.linear_model.Lasso(alpha=0.00001), # Takes a long time, worse than .00005
          # sklearn.linear_model.Lasso(alpha=0.000005), # Takes a really long time, worse than .00001
          # sklearn.linear_model.Lasso(alpha=0.01), # fails
          # sklearn.linear_model.Ridge(alpha=0.0001),
          #sklearn.linear_model.Lasso(alpha=0.0008),
          sklearn.linear_model.Lasso(alpha=0.0006),
          sklearn.linear_model.Lasso(alpha=0.0005),
          sklearn.linear_model.Lasso(alpha=0.0004),
          sklearn.linear_model.Lasso(alpha=0.0003)
          #sklearn.linear_model.Ridge(alpha = 0.1),
          #sklearn.linear_model.ElasticNet(alpha=.0001, l1_ratio=0.5),  # Good! .0145 mean
          # sklearn.linear_model.ElasticNet(alpha=.0001, l1_ratio=0.4),
          #sklearn.linear_model.ElasticNet(alpha=.001, l1_ratio=0.5),  # .0162
          #sklearn.linear_model.ElasticNet(alpha=.001, l1_ratio=0.4),  # .0167
          #sklearn.linear_model.ElasticNet(alpha=.001, l1_ratio=0.6)  # .0146
          #*sklearn.linear_model.ElasticNet(alpha=.001, l1_ratio=0.3), #.0167
          #*sklearn.linear_model.ElasticNet(alpha=.001, l1_ratio=0.4), #  .015
          #*sklearn.linear_model.ElasticNet(alpha=.001, l1_ratio=0.5) # .015
          # sklearn.linear_model.ElasticNet(alpha=.0002, l1_ratio=0.5),
          # sklearn.linear_model.ElasticNet(alpha=.0001, l1_ratio=0.4),
          #sklearn.linear_model.ElasticNet(alpha=.0001, l1_ratio=0.6), # Good! Best?
          #sklearn.linear_model.ElasticNet(alpha=.0001, l1_ratio=0.7) # Good! Best?

          # sklearn.linear_model.ElasticNet(alpha=.00005, l1_ratio=0.5),
          # sklearn.linear_model.ElasticNet(alpha=.0005, l1_ratio=0.5),
          # sklearn.linear_model.ElasticNet(alpha=.00001, l1_ratio=0.25) # Takes very long time
          # sklearn.linear_model.ElasticNet(alpha=.01, l1_ratio=0.5), # fails
          # sklearn.linear_model.ElasticNet(alpha=.001, l1_ratio=0.25),
          # sklearn.linear_model.ElasticNet(alpha=.001, l1_ratio=0.75) # bad result
          ]

In [ ]:
era_split = EraCV(eras = training_data.era)
X, y, era = training_data[feature_cols], training_data.target, training_data.era
for model in models:
    corrs = []
    sharpes = []
    for valid_era in tqdm(range(200,209)):
        train, test = era_split.get_splits(valid_start = valid_era,
                                           valid_n_eras = 4,
                                           train_start = 0,
                                           train_stop = valid_era - 150)
        model.fit(X.iloc[train], y.iloc[train])
        val_preds = model.predict(X.iloc[test])
        eval_df = pd.DataFrame({'prediction':val_preds,
                            'target':y.iloc[test],
                            'era':era.iloc[test]}).reset_index()
        corrs.append(corr(eval_df))
        sharpes.append(sharpe(eval_df))
    print(f'\nmodel: {model.__class__.__name__}')
    if model.__class__.__name__!="LinearRegression":
        print(f'alpha: {model.alpha}')
    if model.__class__.__name__=="ElasticNet":
        print(f'L1 Ratio: {model.l1_ratio}')
    print(f'validation correlations: {corrs}, mean: {np.array(corrs).mean()}')
    print(f'validation sharpes: {sharpes}, mean: {np.array(sharpes).mean()}')


100%|██████████| 9/9 [00:35<00:00,  3.91s/it]

  0%|          | 0/9 [00:00<?, ?it/s]


model: Lasso
alpha: 0.0006
validation correlations: [0.03014910511964791, 0.020066023930782058, 0.023455331495271712, 0.00683277598546162, 0.007698004166776415, 0.011314037489005734, 0.029237444796944085, 0.03258967412588846, 0.008701314275919737], mean: 0.018893745709521972
validation sharpes: [0.7434463679838746, 0.5733117199808012, 0.8088501666389876, 0.22813292627408807, 0.25717342233908236, 0.3262026022078953, 0.8863905844769363, 1.1196655547967, 0.18406534489031698], mean: 0.5696931877320758



100%|██████████| 9/9 [00:34<00:00,  3.79s/it]

  0%|          | 0/9 [00:00<?, ?it/s]


model: Lasso
alpha: 0.0005
validation correlations: [0.029458740894043794, 0.01817598998852242, 0.020780791805464587, 0.004907890584493157, 0.0065858642394740746, 0.010076660981507151, 0.029197870015915478, 0.034526534718563436, 0.009094310924760311], mean: 0.0180894060169716
validation sharpes: [0.7004305367849344, 0.5167690055372463, 0.7048099452932559, 0.15907672864116681, 0.20285032773084904, 0.28438488307263393, 0.8363068169016569, 1.2414054777706072, 0.19224294034779724], mean: 0.5375862957866829



100%|██████████| 9/9 [00:35<00:00,  3.93s/it]

  0%|          | 0/9 [00:00<?, ?it/s]


model: Lasso
alpha: 0.0004
validation correlations: [0.0279975978270083, 0.01584959139434165, 0.017644828277383895, 0.002780866237883827, 0.0035963132276918266, 0.0064540538965367055, 0.026885716534763706, 0.03320744903177996, 0.00824856587997525], mean: 0.01585166470081835
validation sharpes: [0.6697118481097557, 0.4744749117924069, 0.5978362668500886, 0.08680343840298997, 0.11041701692582734, 0.19421810522085706, 0.780371847775435, 1.372086781838461, 0.17570614001928694], mean: 0.4957362618816787



100%|██████████| 9/9 [00:36<00:00,  4.00s/it]


model: Lasso
alpha: 0.0003
validation correlations: [0.026516622597210708, 0.013907756014351794, 0.014699317607767387, 0.0010740341494903313, 0.00197442079728831, 0.004031132750163562, 0.0254930813092623, 0.03174874965824679, 0.006476573489574136], mean: 0.013991298708150591
validation sharpes: [0.6604167392239403, 0.4588178935263151, 0.5156142385316167, 0.03430708843623324, 0.06330095261940913, 0.13080380513877027, 0.7769631761006194, 1.3458658794977612, 0.13253236270839683], mean: 0.4576246817536736


#### Assessing Regression Model Performance

Here we apply the `corr` and `sharpe` methods defined above to predictions made on the validation sample in order to estimate our model's tournament performance.

In [8]:
# Fit final model
X, y, era = training_data[feature_cols], training_data.target, training_data.era
model = sklearn.linear_model.Lasso(alpha=.0005)
model.fit(X, y)
val_sample = training_data.iloc[val_idx]
val_preds = model.predict(val_sample[feature_cols])
eval_df = pd.DataFrame({'prediction':val_preds,
                        'target':val_sample.target,
                        'era':val_sample.era}).reset_index()
val_sharpe = sharpe(eval_df)
val_corr = corr(eval_df)

print((f'The linear regression model\'s validation correlation is {val_corr}. '
       f'Its validation sharpe is {val_sharpe}'))

The linear regression model's validation correlation is 0.02053708106323659. Its validation sharpe is 0.5184176707367818


#### Making Predictions with the Regression Model

In [9]:
ids = []
preds = []

chunksize = 50000

tourn_iter_csv = pd.read_csv(tourn_file, iterator=True, chunksize=1e6)
for chunk in tourn_iter_csv:
    df = chunk[feature_cols]
    out = model.predict(df)
    ids.extend(chunk["id"])
    preds.extend(out)
tourn_iter_csv.close()

In [10]:
linear_regression_predictions_df = pd.DataFrame({
    'id':ids,
    'prediction':preds
})
linear_regression_predictions_df.head()

,id,prediction
0,n0003aa52cab36c2,0.495973
1,n000920ed083903f,0.497192
2,n0038e640522c4a6,0.504312
3,n004ac94a87dc54b,0.500128
4,n0052fe97ea0c05f,0.499820


In [11]:
linear_regression_predictions_df.to_csv("lr_predictions.csv", index=False)

#### Submitting Predictions from the Linear Regression Model

We can use `numerapi` to submit these predictions as follows:

In [12]:
napi.upload_predictions("lr_predictions.csv", model_id=os.environ.get("NUMERAI_MODEL_ID_REGRESSIONS"))

2021-04-10 19:49:41,303 INFO numerapi.base_api: uploading predictions...


'ce552d66-89a5-4b6e-943e-dec5d3691cc8'

In [ ]:
#os.environ.get("NUMERAI_MODEL_ID_REGRESSIONS")